In [14]:
import mediapipe as mp
import os
import csv
import cv2

In [15]:
#Function to Extract Feature from images or Frame
def extract_feature(input_image):
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils 
    image = cv2.imread(input_image)
    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.3) as hands:
        while True:
            results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))
            image_hight, image_width, _ = image.shape
            # Print handedness (left v.s. right hand).
            print(f'Handedness of {input_image}:')
            print(results.multi_handedness)

            # Draw hand landmarks of each hand.
            print(f'Hand landmarks of {input_image}:')
            if not results.multi_hand_landmarks:
                thumb_fingerX = 0
                thumb_fingerY = 0
                index_fingerX = 0
                index_fingerY = 0
                middle_fingerX = 0
                middle_fingerY = 0
                ring_fingerX = 0
                ring_fingerY = 0
                pinky_fingerX = 0
                pinky_fingerY = 0
                return thumb_fingerX, thumb_fingerY, index_fingerX, index_fingerY, middle_fingerX, middle_fingerY, ring_fingerX, ring_fingerY, pinky_fingerX, pinky_fingerY
                break
            
            annotated_image = cv2.flip(image.copy(), 1)
            for hand_landmarks in results.multi_hand_landmarks:
                # Thumb Finger or Jari Jempol
                thumb_fingerX = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * image_width
                thumb_fingerY = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image_width
                # Index Finger or Jari Telunjuk
                index_fingerX = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width
                index_fingerY = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_hight
                # Middle Finger or Jari Tengah
                middle_fingerX = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image_width
                middle_fingerY = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image_width
                # Ring Finger or Jari Manis
                ring_fingerX = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x * image_width
                ring_fingerY = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * image_width
                # Pinky Finger or Jari Kelingking
                pinky_fingerX = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].x * image_width
                pinky_fingerY = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y * image_width
                # Print index finger tip coordinates.
                #print(
                    #f'Index finger tip coordinate: (',
                    #f'{index_fingerX}, '
                    #f'{index_fingerY})')
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            return thumb_fingerX, thumb_fingerY, index_fingerX, index_fingerY, middle_fingerX, middle_fingerY, ring_fingerX, ring_fingerY, pinky_fingerX, pinky_fingerY
            break

In [16]:
#Function to create CSV file or add dataset to the existed CSV file
def toCSV(filecsv, class_type, 
          thumbX, thumbY, 
          indexX, indexY, 
          middleX, middleY, 
          ringX, ringY, 
          pinkyX, pinkyY):
    if os.path.isfile(filecsv):
        print ("File exist thus shall write append to the file")
        with open(filecsv, 'a+', newline='') as file:
            # Create a writer object from csv module
            writer = csv.writer(file)
            writer.writerow([class_type, thumbX, thumbY, indexX, indexY, middleX, middleY, ringX, ringY, pinkyX, pinkyY])
    else:
        print ("File not exist thus shall create new file as", filecsv)
        with open(filecsv, 'w', newline='') as file:
            # Create a writer object from csv module
            writer = csv.writer(file)
            writer.writerow(["class_type", 
                             "thumb_fingerX", "thumb_fingerY", 
                             "index_fingerX", "index_fingerY", 
                             "middle_fingerX", "middle_fingerY", 
                             "ring_fingerX", "ring_fingerY", 
                             "pinky_fingerX", "pinky_fingerY"])
            writer.writerow([class_type, thumbX, thumbY, indexX, indexY, middleX, middleY, ringX, ringY, pinkyX, pinkyY])

In [17]:
#Extract Feature for Training
paths = "SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/training/"
csv_path = "hands_SIBI_training.csv"

if os.path.exists(csv_path):
    print("File does exist, going delete before start extraction and replace it with new")
    os.remove(csv_path)
else:
    print("The file does not exist", csv_path, ",Going Create after Extraction")
    
for dirlist in os.listdir(paths):
    for root, directories, filenames in os.walk(os.path.join(paths, dirlist)):
        print("Inside Folder", dirlist, "Consist :")
        for filename in filenames:
            if filename.endswith(".jpg"):
                print(os.path.join(root, filename), True)
                thumb_fingerX, thumb_fingerY, index_fingerX, index_fingerY, middle_fingerX, middle_fingerY, ring_fingerX, ring_fingerY, pinky_fingerX, pinky_fingerY = extract_feature(os.path.join(root, filename))
                print(index_fingerX, index_fingerY)
                if ((not index_fingerX == 0) and (not index_fingerY == 0)):
                    toCSV(csv_path, dirlist, 
                          thumb_fingerX, thumb_fingerY, 
                          index_fingerX, index_fingerY, 
                          middle_fingerX, middle_fingerY, 
                          ring_fingerX, ring_fingerY, 
                          pinky_fingerX, pinky_fingerY)
                else :
                    print("Hand does not have landmarks")
                print("\n")
                

File does exist, going delete before start extraction and replace it with new
Inside Folder A Consist :
SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/training/A\A (1).jpg True
Handedness of SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/training/A\A (1).jpg:
[classification {
  index: 1
  score: 0.9999857
  label: "Right"
}
]
Hand landmarks of SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/training/A\A (1).jpg:
910.5516672134399 867.297887802124
File not exist thus shall create new file as hands_SIBI_training.csv


SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/training/A\A (10).jpg True
Handedness of SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/training/A\A (10).jpg:
[classification {
  index: 1
  score: 0.9999713
  label: "Right"
}
]
Hand landmarks of SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/training/A\A (10).jpg:
887.608528137207 981.4499020576477
File exist thus shall write append to the file


SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/training/A\A (11).jpg True
Handedness of SIBI_datasets_LEMLITBANG_S

In [18]:
#Extract Feature for Validation
paths = "SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/validation/"
csv_path = "hands_SIBI_validation.csv"

if os.path.exists(csv_path):
    print("File does exist, going delete before start extraction and replace it with new")
    os.remove(csv_path)
else:
    print("The file does not exist", csv_path, ",Going Create after Extraction")

for dirlist in os.listdir(paths):
    for root, directories, filenames in os.walk(os.path.join(paths, dirlist)):
        print("Inside Folder", dirlist, "Consist :")
        for filename in filenames:
            if filename.endswith(".jpg"):
                print(os.path.join(root, filename), True)
                thumb_fingerX, thumb_fingerY, index_fingerX, index_fingerY, middle_fingerX, middle_fingerY, ring_fingerX, ring_fingerY, pinky_fingerX, pinky_fingerY = extract_feature(os.path.join(root, filename))
                print(index_fingerX, index_fingerY)
                if ((not index_fingerX == 0) and (not index_fingerY == 0)):
                    toCSV(csv_path, dirlist, 
                          thumb_fingerX, thumb_fingerY, 
                          index_fingerX, index_fingerY, 
                          middle_fingerX, middle_fingerY, 
                          ring_fingerX, ring_fingerY, 
                          pinky_fingerX, pinky_fingerY)
                else :
                    print("Hand does not have landmarks")
                print("\n")

File does exist, going delete before start extraction and replace it with new
Inside Folder A Consist :
SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/validation/A\A (1).jpg True
Handedness of SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/validation/A\A (1).jpg:
[classification {
  index: 1
  score: 0.9999907
  label: "Right"
}
]
Hand landmarks of SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/validation/A\A (1).jpg:
613.4974956512451 1111.3165616989136
File not exist thus shall create new file as hands_SIBI_validation.csv


SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/validation/A\A (2).jpg True
Handedness of SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/validation/A\A (2).jpg:
[classification {
  index: 1
  score: 0.9999994
  label: "Right"
}
]
Hand landmarks of SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/validation/A\A (2).jpg:
663.4631156921387 1154.3641090393066
File exist thus shall write append to the file


Inside Folder B Consist :
SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/validation/B\B (1).jpg Tru